In [ ]:
!pip install ortools

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from itertools import product

import sys
sys.path.append("../lecture6/")

import simplicial
import simplicial.drawing
from simplicialx.simplicial import SimplicialComplex

from ortools.linear_solver import pywraplp

np.set_printoptions(precision=2, linewidth=100, suppress=True)

In [ ]:
def draw_simpicial_complex():
    
    plt.figure(figsize=(6.5,4.2))
    plt.scatter(X[:,0], X[:,1], c="k", s=25)
    plt.xlim(-0.5, 2)
    plt.ylim(-0.25, 1.25)
    
    plt.box(False)
    plt.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='off')

    # edges
    for edge in cmplx.simplices[1]:
        plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=0.5, linewidth=1)

    # triangles
    for triangle in cmplx.simplices[2]:
        t = plt.Polygon(X[triangle], color="b", alpha=0.1, linewidth=0)
        plt.gca().add_patch(t)

    for vertex in cmplx.simplices[0]:
        plt.annotate(vertex[0], (X[vertex,0]-0.2, X[vertex,1]-0.1), fontsize=12)

    plt.show()

In [ ]:
X = np.array([
    [0.0, 0.5],
    [0.5, 1.0],
    [1.5, 1.0],
    [1.5, 0.0],
    [0.5, 0.0],
])

In [ ]:
cmplx = SimplicialComplex()

cmplx.add([0, 1])
cmplx.add([0, 4])
cmplx.add([1, 2, 4])
cmplx.add([2, 3, 4])

cmplx.simplices

<img style="float: left;" src="images/K.png">

### Spaces of simplices

Given a simplicial complex $K = \{\emptyset, 0, 1, 2, 3, 4, 01, 04, 12, 14, 23, 24, 34, 124, 234 \}$ its spaces of simplices are

$$
\begin{align}
\Sigma_0 &= \{0, 1, 2, 3\}\\
\Sigma_1 &= \{01, 04, 12, 14, 23, 24, 34 \}\\
\Sigma_2 &= \{124, 234 \}
\end{align}
$$

### Chain spaces

Chain space $C_k$ is defined as a formal sum of $k$-simplicies with coefficients in a field $\mathbb{k}$

$$c_k = \sum_i \alpha \sigma_i \in C_k,~\textrm{where}~\sigma_i \in \Sigma_k, \alpha \in \mathbb{k}$$

#### Examples

Vertex chains $C_0$

$$
c_0 = [0] \sim [0 * 1 + 0 * 2 + 0 * 3 + 0 * 4]\\
c_0' = [1 + 2 + 4] \sim [1 * 1 + 1 * 2 + 0 * 3 + 1 * 4]
$$

Edge chains $C_1$

$$
c_1 = [12 + 23 + 34]\\
c_1' = [01 + 23 + 24 + 34]
$$

Triangle chains $C_2$

$$
c_2 = [124]\\
c_2' = [234 + 234]\\
$$

#### Summation

One can take sums of chains, over $\mathbb{Z}_2$ the summation is defined modulo 2

$$
\begin{align}
c_1 + c_1' &= [12 + 23 + 34] + [12 + 14 + 24 + 34]\\
&= [14 + 23 + 24]
\end{align}
$$

In [ ]:
# 01, 04, 12, 14, 23, 24, 34
c1 = np.array([0, 0, 1, 0, 1, 0, 1])
c1_prime = np.array([0, 0, 1, 1, 0, 1, 1])
(c1 + c1_prime) % 2

<img style="float: left;" src="images/K.png">

### Boundary operator

Given a $k$-simplex $\sigma = [v_0, v_1, \dots, v_k] \in \Sigma_k$ its boundary $\partial_k \sigma \in C_{k-1}$ is defined

$$\partial_k \sigma = \sum_{i=0}^k~[v_0, v_1, \dots, v_{i-1}, v_{i+1}, \dots, v_k]$$

#### Examples

Edge

$$c_1 = 12\\
\partial c_1 = 2 + 1$$

Triangle

$$c_2 = 234\\
\partial c_2 = 34 + 24 + 23$$

#### Matrix representation

Boundary operator $\partial_k$ for a fixed basis can be represented by a matrix $\mathbf{B}_k$ having $k$-simplices $\sigma$ on columns and $k-1$-simplices $\tau$ on rows and $b_{ij} = 1$ if $\tau$ is in a boundary of $\sigma$.

In [ ]:
# 01, 04, 12, 14, 23, 24, 34 - columns
# 0, 1, 2, 3, 4 - rows
B1 = np.abs(cmplx.boundary_operator_matrix(k=1)).astype(int)
B1

In [ ]:
cmplx.boundary_operator_matrix(k=2)

In [ ]:
# 124, 234 - columns
# 01, 04, 12, 14, 23, 24, 34 - rows
B2 = np.abs(cmplx.boundary_operator_matrix(k=2)).astype(int)
B2

### Boundaries of chains

#### Boundary of a chain

Boundary operator is linear

$$
\begin{align}
c &= 01 + 12 + 23\\\\
\partial c &= \partial(01 + 12 + 23)\\
\partial c &= \partial(01) + \partial(12) + \partial(23)\\
\partial c &= (1 + 0) + (2 + 1) + (3 + 2)\\
\partial c &= 0 + 3
\end{align}
$$

In [ ]:
c = np.array([1, 0, 1, 0, 1, 0, 0]) # 1*01 + 0*04 + 1*12 + 0*14 + 1*23 + 0*24 + 0*34
B1 @ c % 2

<img style="float: left;" src="images/K.png">

#### Boundary of a cycle

$$
\begin{align}
z &= 01 + 12 + 24 + 04\\\\
\partial z &= \partial(01) + \partial(12) + \partial(24) + \partial(04)\\
\partial z &= (1 + 0) + (2 + 1) + (4 + 2) + (4 + 0)\\
\partial z &= 0
\end{align}
$$

In [ ]:
z = np.array([1, 1, 1, 0, 0, 1, 0]) # 01, 04, 12, 14, 23, 24, 34
B1 @ z % 2

<img style="float: left;" src="images/K.png">

#### Exercise

Find the boundary of the other cycle in the complex $K$.

In [ ]:
z_prime = np.array([0, 0, 0, 0, 0, 0, 0]) # 01, 04, 12, 14, 23, 24, 34
boundary = B1 @ z_prime % 2 # check if z_prime boundary is zero
boundary

### Chain complex

Chain complex is a sequence of chain spaces connected with boundary maps

$$C_2 \xrightarrow{\partial_2} C_1  \xrightarrow{\partial_1} C_0$$

For a given chain space $C_k$ the space of cycles $Z_k$ consists of all elements of $C_k$ which boundary $\partial_k$ is zero.

$$
\begin{align}
Z_k &= \mathrm{ker}~\partial_k\\
Z_k &= \{ c \in C_k \mid \partial_k = 0 \}\\\\
B_k &= \mathrm{im}~\partial_{k+1}\\
B_k &= \{ c \in C_k \mid \partial_{k+1}d = c,~\mathrm{for~some}~d \in C_{k+1} \}
\end{align}
$$

Hierarchy of spaces

$$B_k \subseteq Z_k \subseteq C_k$$

<img style="float: left;" src="images/K.png">

#### 1-cycles and 1-boundaries of $K$

$$Z_1 = \{ [01 + 04 + 14], [12 + 14 + 24], [23 + 24 + 34], [12 + 14 + 24 + 34], [01 + 04 + 12 + 24], [01 + 04 + 12 + 23 + 34] \}$$
$$B_1 = \{[12 + 14 + 24], [23 + 24 + 34], [12 + 14 + 24 + 34]\}$$

### Homologous cycles

Two cycles $z, z' \in C_k$ are said homologous $z \sim z'$ if their difference $z - z' \in B_k$.

In [ ]:
# 01, 04, 12, 14, 23, 24, 34
z = np.array([1, 1, 1, 0, 0, 1, 0]) # 01 + 04 + 12 + 24
z_prime1 = np.array([1, 1, 0, 1, 0, 0, 0]) # 01 + 04 + 14
z_prime2 = np.array([1, 1, 1, 0, 1, 0, 1]) # 01 + 04 + 12 + 23 + 34

In [ ]:
(z - z_prime1) % 2 # 12 + 14 + 24

In [ ]:
(z - z_prime2) % 2 # 23 + 24 + 34

### Optimal representatives

In [ ]:
X = np.array([
    [0.0, 0.5],
    [0.5, 1.0],
    [1.5, 1.0],
    [2.5, 1.0],
    [2.5, 0.0],
    [1.5, 0.0],
    [0.5, 0.0],
])

In [ ]:
cmplx = SimplicialComplex()

cmplx.add([0, 1])
cmplx.add([0, 6])
cmplx.add([2, 3])
cmplx.add([3, 4])
cmplx.add([4, 5])
cmplx.add([1, 2, 6])
cmplx.add([2, 5, 6])

cmplx.simplices

In [ ]:
B1 = cmplx.boundary_operator_matrix(k=1).astype(int) # edge-vertices
B2 = cmplx.boundary_operator_matrix(k=2) # triangle-edges

#### Homology representative

In [ ]:
x_init = np.array([1, -1, 1, 0, 0, 0, 1, 0, 0, 0]) # 01, 60, 12, 26

In [ ]:
plt.figure(figsize=(9,6))
plt.title("Initial cycle")
plt.scatter(X[:,0], X[:,1], c="k", s=25)
plt.xlim(-0.5, 3)
plt.ylim(-0.5, 1.5)

# edges
for edge in cmplx.simplices[1]:
    plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=0.5, linewidth=1)

# triangles
for triangle in cmplx.simplices[2]:
    t = plt.Polygon(X[triangle], color="b", alpha=0.1, linewidth=0)
    plt.gca().add_patch(t)

# init cycle
for i, edge in enumerate(cmplx.simplices[1]):
    plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=abs(x_init[i]), linewidth=2.5*abs(x_init[i]), label="({}, {}): {:.2f}".format(edge[0], edge[1], x_init[i]))

for vertex in cmplx.simplices[0]:
    plt.annotate(vertex[0], (X[vertex,0]-0.2, X[vertex,1]-0.1), fontsize=12)

plt.legend(loc="upper right", bbox_to_anchor=(1.25, 1.015))
plt.show()


### Naive algorithm

Check for all cycles in $Z_k$ where their difference is an element (or combination) of $B_k$. Choose one minimizing desired objective function.

#### Find the shortest cycle homologous to $x_{\mathrm{init}}$

In [ ]:
x_init = np.array([1, -1, 1, 0, 0, 0, 1, 0, 0, 0]) # 01, 60, 12, 26

In [ ]:
cycles_loss = []

for w in product([-1, 0, 1], repeat=B2.shape[-1]):
    coeffs = np.asarray(w)
    boundaries = (B2 @ np.asarray(w)).astype(int)
    x_homologous = (x_init + B2 @ np.asarray(w)).astype(int)
    loss = np.linalg.norm(x_homologous, ord=1)
    cycles_loss.append((x_homologous, loss))
    print(coeffs, boundaries, x_homologous, loss)

#### Optimal cycle

In [ ]:
x_opt = sorted(cycles_loss, key=lambda x: x[1])[0][0]
x_opt, np.linalg.norm(x_opt, ord=1)

#### Check whether optimal cycle is a cycle

In [ ]:
B1 @ x_opt

In [ ]:
plt.figure(figsize=(9,6))
plt.title("Initial and optimal cycles")
plt.scatter(X[:,0], X[:,1], c="k", s=25)
plt.xlim(-0.5, 3)
plt.ylim(-0.5, 1.5)

# edges
for edge in cmplx.simplices[1]:
    plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=0.5, linewidth=1)

# triangles
for triangle in cmplx.simplices[2]:
    t = plt.Polygon(X[triangle], color="b", alpha=0.1, linewidth=0)
    plt.gca().add_patch(t)

# init cycle
for i, edge in enumerate(cmplx.simplices[1]):
    plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=abs(x_init[i]), linewidth=5*abs(x_init[i]), label="({}, {}): {:.2f}".format(edge[0], edge[1], x_init[i]))
    
# optimal cycle
for i, edge in enumerate(cmplx.simplices[1]):
    plt.plot(X[edge][:,0], X[edge][:,1], c="r", alpha=abs(x_opt[i]), linewidth=2.5*abs(x_opt[i]), label="({}, {}): {:.2f}".format(edge[0], edge[1], x_opt[i]))

for vertex in cmplx.simplices[0]:
    plt.annotate(vertex[0], (X[vertex,0]-0.2, X[vertex,1]-0.1), fontsize=12)

plt.legend(loc="upper right", bbox_to_anchor=(1.25, 1.015))
plt.show()


### Linear programming

In [ ]:
n_edges = B1.shape[-1]
n_triangles = B2.shape[-1]

# init solver
solver = pywraplp.Solver.CreateSolver("CLP_LINEAR_PROGRAMMING")
inf = solver.infinity()

# init optimization variables w/ bounds
w = {}
for i in range(n_triangles):
    w[i] = solver.IntVar(-inf, inf, "w[{}]".format(i))

x, x_pos, x_neg = {}, {}, {}
for i in range(n_edges):
    x[i] = solver.IntVar(-inf, inf, "x[{}]".format(i))
    x_pos[i] = solver.IntVar(0, inf, "x_+[{}]".format(i))
    x_neg[i] = solver.IntVar(0, inf, "x_-[{}]".format(i))
    
# add constraints
for i in range(n_edges):
    solver.Add(x[i] == (x_init[i] + sum([w[j] * B2[i,j] for j in range(n_triangles)])))
    
for i in range(len(x_init)):
    solver.Add(x[i] == (x_pos[i] - x_neg[i]))
    
# add objective function
f = sum([x_pos[i] + x_neg[i] for i in range(len(x_init))])
solver.Minimize(f)

# solve
solver.Solve()

#### Optimal cycle

In [ ]:
x_opt = np.array([x_i.solution_value() for i, x_i in x.items()]).astype(int)
x_opt, np.linalg.norm(x_opt, ord=1)

In [ ]:
plt.figure(figsize=(9,6))
plt.title("Initial and optimal cycles")
plt.scatter(X[:,0], X[:,1], c="k", s=25)
plt.xlim(-0.5, 3)
plt.ylim(-0.5, 1.5)

# edges
for edge in cmplx.simplices[1]:
    plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=0.5, linewidth=1)

# triangles
for triangle in cmplx.simplices[2]:
    t = plt.Polygon(X[triangle], color="b", alpha=0.1, linewidth=0)
    plt.gca().add_patch(t)

# init cycle
for i, edge in enumerate(cmplx.simplices[1]):
    plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=abs(x_init[i]), linewidth=5*abs(x_init[i]), label="({}, {}): {:.2f}".format(edge[0], edge[1], x_init[i]))
    
# optimal cycle
for i, edge in enumerate(cmplx.simplices[1]):
    plt.plot(X[edge][:,0], X[edge][:,1], c="r", alpha=abs(x_opt[i]), linewidth=2.5*abs(x_opt[i]), label="({}, {}): {:.2f}".format(edge[0], edge[1], x_opt[i]))

for vertex in cmplx.simplices[0]:
    plt.annotate(vertex[0], (X[vertex,0]-0.2, X[vertex,1]-0.1), fontsize=12)

plt.legend(loc="upper right", bbox_to_anchor=(1.25, 1.015))
plt.show()


## Harmonic representatives

#### Higher-order Laplacian

Given a simplicial complex $K$ with boundary matrices $\{\mathbf{B}_k\}_{k=1}^{\dim K}$, the $k$-th higher-order Laplacian is defined

\begin{equation}
\mathbf{L}_k = \mathbf{B}_k^T \mathbf{B}_k + \mathbf{B}_{k+1} \mathbf{B}_{k+1}^T,
\end{equation}

then the $k$-the Betti number of $K$ is the multiplicity of zero eigenvalues of $\mathbf{L}_k$ and solutions of $\mathbf{L}_k \mathbf{v} = \mathbf{0}$ are harmonic representatives of $k$-th homology classes.

### 0-dim homology representatives

In [ ]:
cmplx = SimplicialComplex()

cmplx.add([0, 1])
cmplx.add([2, 3])
cmplx.add([3, 4])
cmplx.add([4, 5])
cmplx.add([2, 5, 6])

cmplx.simplices

In [ ]:
plt.figure(figsize=(9,6))
plt.scatter(X[:,0], X[:,1], c="k", s=25)
plt.xlim(-0.5, 3)
plt.ylim(-0.5, 1.5)

# edges
for edge in cmplx.simplices[1]:
    plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=0.5, linewidth=1)

# triangles
for triangle in cmplx.simplices[2]:
    t = plt.Polygon(X[triangle], color="b", alpha=0.1, linewidth=0)
    plt.gca().add_patch(t)
    
for vertex in cmplx.simplices[0]:
    plt.annotate(vertex[0], (X[vertex,0]-0.2, X[vertex,1]-0.1), fontsize=12)

plt.show()

#### Laplacian $L_0$

In [ ]:
B1 = cmplx.boundary_operator_matrix(k=1)

L0 = B1 @ B1.T

In [ ]:
eigenvalues, eigenvectors = np.linalg.eigh(L0)
eigenvalues, eigenvectors

In [ ]:
# for each zero eigenvalue
for k in range(len(eigenvalues)):
    
    if np.isclose(eigenvalues[k], 0):

        plt.figure(figsize=(9,6))
        plt.title("{}-th eigenvalue".format(k))
        plt.scatter(X[:,0], X[:,1], c="k", s=25)
        plt.xlim(-0.5, 3)
        plt.ylim(-0.5, 1.5)

        # edges
        for edge in cmplx.simplices[1]:
            plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=0.5, linewidth=1)

        # triangles
        for triangle in cmplx.simplices[2]:
            t = plt.Polygon(X[triangle], color="b", alpha=0.1, linewidth=0)
            plt.gca().add_patch(t)

        
        # harmonic representatives
        eigenvector_normalized = np.abs(eigenvectors[:,k]) / np.abs(eigenvectors[:,k]).max()
        for i, vertex in enumerate(cmplx.simplices[0]):
            plt.scatter(X[i,0], X[i,1], c="r", s=100*abs(eigenvector_normalized[i]), label="({}, {}): {:.2f}".format(edge[0], edge[1], eigenvectors[i,k]))

        for vertex in cmplx.simplices[0]:
            plt.annotate(vertex[0], (X[vertex,0]-0.2, X[vertex,1]-0.1), fontsize=12)

        plt.legend(loc="upper right", bbox_to_anchor=(1.35, 1.0))
        plt.show()


### 1-dim homology representatives

In [ ]:
cmplx = SimplicialComplex()

cmplx.add([0, 1])
cmplx.add([0, 6])
cmplx.add([2, 3])
cmplx.add([3, 4])
cmplx.add([4, 5])
cmplx.add([1, 2, 6])
cmplx.add([2, 5, 6])

cmplx.simplices

In [ ]:
plt.figure(figsize=(9,6))
plt.scatter(X[:,0], X[:,1], c="k", s=25)
plt.xlim(-0.5, 3)
plt.ylim(-0.5, 1.5)

# edges
for edge in cmplx.simplices[1]:
    plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=0.5, linewidth=1)

# triangles
for triangle in cmplx.simplices[2]:
    t = plt.Polygon(X[triangle], color="b", alpha=0.1, linewidth=0)
    plt.gca().add_patch(t)
    
for vertex in cmplx.simplices[0]:
    plt.annotate(vertex[0], (X[vertex,0]-0.2, X[vertex,1]-0.1), fontsize=12)

plt.show()

#### Laplacian $L_1$

In [ ]:
B1 = cmplx.boundary_operator_matrix(k=1)
B2 = cmplx.boundary_operator_matrix(k=2)

L1 = B1.T @ B1 + B2 @ B2.T

In [ ]:
eigenvalues, eigenvectors = np.linalg.eigh(L1)
eigenvalues, eigenvectors

In [ ]:
# for each zero eigenvalue
for k in range(len(eigenvalues)):
    
    if np.isclose(eigenvalues[k], 0):

        plt.figure(figsize=(9,6))
        plt.title("{}-th eigenvalue".format(k))
        plt.scatter(X[:,0], X[:,1], c="k", s=25)
        plt.xlim(-0.5, 3)
        plt.ylim(-0.5, 1.5)

        # edges
        for edge in cmplx.simplices[1]:
            plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=0.1, linewidth=1)

        # triangles
        for triangle in cmplx.simplices[2]:
            t = plt.Polygon(X[triangle], color="b", alpha=0.1, linewidth=0)
            plt.gca().add_patch(t)

        # harmonic representatives
        eigenvector_normalized = np.abs(eigenvectors[:,k]) / np.abs(eigenvectors[:,k]).max()
        for i, edge in enumerate(cmplx.simplices[1]):
            plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=eigenvector_normalized[i], linewidth=8*eigenvector_normalized[i], label="({}, {}): {:.2f}".format(edge[0], edge[1], eigenvectors[i,k]))

        for vertex in cmplx.simplices[0]:
            plt.annotate(vertex[0], (X[vertex,0]-0.2, X[vertex,1]-0.1), fontsize=12)

        plt.legend(loc="upper right", bbox_to_anchor=(1.29, 1.015))
        plt.show()
